In [76]:
import numpy as np
import pandas as pd

TOLERANCE = 10e-6

np.linalg.inv(np.matrix([[18, -8], [-8, 19.75]]))
np.linalg.inv(np.matrix([[19.75]]))

B_mat = np.matrix([[-22, 10, 12], [10, -18, 8], [12, 8, -19.75]])
B_P = np.matrix([[18, -8], [-8, 19.75]])
B_Q = np.matrix([[19.75]])

B_P_inv = np.linalg.inv(B_P)
B_Q_inv = np.linalg.inv(B_Q)


# converts actual index to zero-based
def B(row, col):
    return B_mat[row - 1, col - 1]


V_2 = 0.98
P_2 = 1
P_3 = -2
Q_3 = -0.5


def has_converged(voltage_list):
    cur = Vs[-1]
    prev = Vs[-2] if len(Vs) > 1 else 0  # dummy value at start

    if np.abs(cur - prev) > TOLERANCE:
        return False

    return True


# flat start
# [[delta2, delta3],...]
deltas = [np.array([0, 0])]
# [[V_3]...]
Vs = [np.array([1])]


i = 0
while not has_converged(Vs) and i < 1000:
    i += 1
    delta_2 = deltas[-1][0]
    delta_3 = deltas[-1][1]
    v_3 = Vs[-1][0]

    # delta calculations
    matrix_A1 = B_P_inv.dot(np.linalg.inv(np.diag([V_2, v_3])))

    mismatch_P2 = P_2 - (
        V_2 * (B(2, 1) * np.sin(delta_2) + v_3 * B(2, 3) * np.sin(delta_2 - delta_3))
    )
    mismatch_P3 = P_3 - (
        v_3 * (B(3, 1) * np.sin(delta_2) + V_2 * B(3, 2) * np.sin(delta_3 - delta_2))
    )

    mismatch_P = np.matrix([[mismatch_P2], [mismatch_P3]])
    deltas_new = np.add(deltas[-1].reshape(2, 1), matrix_A1.dot(mismatch_P))

    # voltage calculations
    matrix_A2 = B_Q_inv.dot(np.linalg.inv(np.diag([v_3])))
    mismatch_Q3 = Q_3 - (
        -v_3
        * (
            B(3, 1) * np.cos(delta_3)
            + B(3, 2) * V_2 * np.cos(delta_3 - delta_2)
            + B(3, 3) * v_3
        )
    )

    mismatch_Q = np.matrix([[mismatch_Q3]])

    v_new = np.add(Vs[-1].reshape(1, 1), matrix_A2.dot(mismatch_Q))

    deltas.append(np.ravel(deltas_new))
    Vs.append(np.ravel(v_new))


df = pd.DataFrame(
    {
        "iteration": np.arange(1, i + 1),
        "angle for bus 2 [radians]": [xx[0] for xx in deltas[1:]],
        "angle for bus 3 [radians]": [xx[1] for xx in deltas[1:]],
        "magnitude for bus 3[pu]": [xx[0] for xx in Vs[1:]],
    }
)

df


,iteration,angle for bus 2 [radians],angle for bus 3 [radians],magnitude for bus 3[pu]
0,1,0.014247,-0.095495,0.979241
1,2,-0.021580,-0.178600,0.973548
2,3,-0.035969,-0.213260,0.964004
3,4,-0.041769,-0.228797,0.958309
4,5,-0.044441,-0.236369,0.955383
5,6,-0.045762,-0.240184,0.953875
6,7,-0.046428,-0.242129,0.953096
7,8,-0.046768,-0.243125,0.952694
8,9,-0.046942,-0.243638,0.952486
9,10,-0.047032,-0.243902,0.952379
